In [1]:
# use remote s3 datasets instead of locally stored tmy h5 file
#!pip install pyspark
%env hs_endpoint = 'https://developer.nrel.gov/api/hsds'
%env hs_api_key = 'tmhsEBYW3KWbaOyMWECph51LOGnVIAcOxEAzbZgA'
%env hs_username = None
%env hs_password = None

import h5pyd
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from scipy.spatial import cKDTree
import csv
from pyspark.sql.functions import col
from pyspark.sql import *

def load_data():
    # Loads the data for this question. Do not change this function.
    # This function should only be called with the parameter 'small' or 'large'

    input_bucket = "s3://nrel-pds-nsrdb/v3/tmy"
    
    # Load TMY Data
    # file_path = '/nsrdb_tmy-2020.h5'
    file_path = "/nrel/nsrdb/v3/nsrdb_2017.h5"
    #tmy = spark.read.csv(input_bucket + file_path,header=True,inferSchema=True)
    #print("TMY Count: ", trips.count())  # Prints # of trips (# of records, as each record is one trip)
    
    with h5pyd.File(file_path, mode='r') as f:
        meta = pd.DataFrame(f['meta'][...])    

    return tmy


env: hs_endpoint='https://developer.nrel.gov/api/hsds'
env: hs_api_key='tmhsEBYW3KWbaOyMWECph51LOGnVIAcOxEAzbZgA'
env: hs_username=None
env: hs_password=None


In [2]:
# Open the desired year of nsrdb data
# server endpoint, username, password is found via a config file

#tmy = load_data()

f = h5py.File('dataset/tmy2020.h5', 'r')
list(f.keys())


['air_temperature',
 'alpha',
 'aod',
 'asymmetry',
 'cld_opd_dcomp',
 'cld_reff_dcomp',
 'clearsky_dhi',
 'clearsky_dni',
 'clearsky_ghi',
 'cloud_press_acha',
 'cloud_type',
 'coordinates',
 'dew_point',
 'dhi',
 'dni',
 'fill_flag',
 'ghi',
 'meta',
 'ozone',
 'relative_humidity',
 'solar_zenith_angle',
 'ssa',
 'surface_albedo',
 'surface_pressure',
 'time_index',
 'tmy_year',
 'tmy_year_short',
 'total_precipitable_water',
 'wind_direction',
 'wind_speed']

In [4]:
# Helper functions for calculating wind power density

# Mathematical constants
Rv = 461.4964
Rd = 287.0531
Eso = 6.1078
c0 = 0.99999683
c1 = -0.90826951e-2
c2 = 0.78736169e-4
c3 = -0.61117958e-6
c4 = 0.43884187e-8
c5 = -0.29883885e-10
c6 = 0.21874425e-12
c7 = -0.17892321e-14
c8 = 0.11112018e-16
c9 = -0.30994571e-19


def CalculateAirDensity(temp, press, dew_point):
    """
    Function for calculating the density of humidified air.
    :param temp: ambient temperature in degrees Celsius
    :param press: ambient air pressure in hPa
    :param dew_point: dew point in degrees Celsius
    :return: air density in kg/m3
    Reference: All constants and formulas were taken from
               https://www.gribble.org/cycling/air_density.html
    """
    # Calculate pressure of water vapor
    p = c0 + dew_point * \
        (c1 + dew_point * \
        (c2 + dew_point * \
        (c3 + dew_point * \
        (c4 + dew_point * \
        (c5 + dew_point * \
        (c6 + dew_point * \
        (c7 + dew_point * \
        (c8 + dew_point * \
        (c9)))))))))

    press_water_vapor = Eso / (p ** 8)
    
    # Calculate pressure of dry air
    press_dry_air = press - press_water_vapor
    
    # Convert air temperature from Celcius to Kelvins
    temp_K = temp + 273.15
    
    # Calculate air density
    return ((press_dry_air / (Rd * temp_K)) + (press_water_vapor / (Rv * temp_K))) * 100


def WindPowerDensity(wind_speed, temp, press, dew_point):
    """
    Function to calculate the theoretical wind power density according to:
    [12] H. Cetinay, F. A. Kuipers, and A. N. Guven, “Optimal siting and sizing of wind farms,”
         Renewable Energy, 101, 51-58, 2017.
    :param wind_speed: wind speed
    :param temp: ambient temperature in degrees Celsius
    :param press: ambient air pressure in hPa
    :param dew_point: dew point in degrees Celsius
    :return:
    """
    return 0.5 * CalculateAirDensity(temp, press, dew_point) * (wind_speed**3)

In [6]:
def nearest_site(tree, lat_coord, lon_coord):
    lat_lon = np.array([lat_coord, lon_coord])
    dist, pos = tree.query(lat_lon)
    return pos

In [7]:
dset_coords = f['coordinates'][...]

dset_names = \
['air_temperature',
 'alpha',
 'aod',
 'asymmetry',
 'cld_opd_dcomp',
 'cld_reff_dcomp',
 'clearsky_dhi',
 'clearsky_dni',
 'clearsky_ghi',
 'cloud_press_acha',
 'cloud_type',
 'dew_point',
 'dhi',
 'dni',
 'ghi',
 'ozone',
 'relative_humidity',
 'solar_zenith_angle',
 'ssa',
 'surface_albedo',
 'surface_pressure',
 'total_precipitable_water',
 'wind_direction',
 'wind_speed']

def agg_on_coord(ploc):
    ploc_idx = nearest_site(tree, ploc[0], ploc[1] )
    #print(ploc_idx)
    #print(dset_coords[ploc_idx])

    # df = pd.DataFrame(f['meta'][...])
    # df['ghi'] = data / dset.attrs['psm_scale_factor']

    dict_result = {}

    dict_result['adj_latitude'] = dset_coords[ploc_idx][0]
    dict_result['adj_longitude'] = dset_coords[ploc_idx][1]

    # get mean of yearly aggregated fields
    for field in dset_names:
#        print(field)
        dset = f[field]
        dict_result['avg_' + field] = np.mean(dset[::1,ploc_idx] / dset.attrs['psm_scale_factor'])

    # get std dev of yearly aggregated fields
    for field in dset_names:
        dset = f[field]
        dict_result['std_' + field] = np.std(dset[::1,ploc_idx] / dset.attrs['psm_scale_factor'])

    df = pd.DataFrame([dict_result])
    return df

# df = agg_on_coord((44.3142,-89.8964))
# pd.set_option('display.max_columns', None)
# df.head()

In [8]:
def which_bucket(value, data_type):
    """
    Helper function to determine which bucket a given wind speed or temperature falls into.
    :value: the value to test
    :data: whether the value is 'wind' or 'temp'
    :return: the list index the value falls into 
    """
    if data_type == 'wind':
        if value < 5.0:
            return 0
        elif value >= 5.0 and value < 10.0:
            return 1
        elif value >= 10.0 and value < 15.0:
            return 2      
        elif value >= 15.0 and value < 20.0:
            return 3        
        elif value >= 20.0 and value < 25.0:
            return 4 
        elif value >= 25.0:
            return 5
        
    elif data_type == 'temp':
        if value < 0.0:
            return 0
        elif value >= 0.0 and value < 10.0:
            return 1
        elif value >= 10.0 and value < 20.0:
            return 2      
        elif value >= 20.0 and value < 30.0:
            return 3        
        elif value >= 30.0 and value < 40.0:
            return 4 
        elif value >= 40.0:
            return 5
    

In [9]:
def get_additional_variables(ploc):
    """
    Helper function to calculate wind and temperature bands and wind power density for a single plant.
    :ploc: tuple containing the latitude and longitude of the desired power plant
    :return: pandas dataframe containing wind and temperature bands and wind power density for a single plant    
    """
    
    # Create file objects for accessing needed datasets in the HDF5 file
    dset_coords = f['coordinates'][...]
    dset_temp = f['air_temperature']
    dset_press = f['surface_pressure']
    dset_dew_point = f['dew_point']
    dset_wind_speed = f['wind_speed']
    
    # Get the index corresponding to the desired plant
    ploc_idx = nearest_site(tree, ploc[0], ploc[1])
    
    # Load the needed data into 1d numpy arrays
    np_temp = dset_temp[:, ploc_idx]
    np_press = dset_press[:, ploc_idx]
    np_dew_point = dset_dew_point[:, ploc_idx]
    np_wind_speed = dset_wind_speed[:, ploc_idx]
    
    # Scale each array
    np_temp = np_temp[:] / dset_temp.attrs['psm_scale_factor']
    np_press = np_press[:] / dset_press.attrs['psm_scale_factor']
    np_dew_point = np_dew_point[:] / dset_dew_point.attrs['psm_scale_factor']
    np_wind_speed = np_wind_speed[:] / dset_wind_speed.attrs['psm_scale_factor']
    
    # Create an empty dictionary to hold the results
    dict_result = {}
    
    # Add the actual latitude and longitude of the weather station
    dict_result['adj_latitude'] = dset_coords[ploc_idx][0]
    dict_result['adj_longitude'] = dset_coords[ploc_idx][1]
    
    
    # Create a list to hold the wind band data
    wind_buckets = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    
    # Add wind band data
    for i in range(len(np_wind_speed)):
        bucket = which_bucket(np_wind_speed[i], 'wind')
        wind_buckets[bucket] = wind_buckets[bucket] + 1
                   
    # Find the total number of hours in the year
    total_points = sum(wind_buckets)
    
    # Check if any values did not fall into a bucket
    if total_points != 8760:
        print('ploc', ploc, ' total_wind_points: ', total_points)
        
    
                   
    # Calculate the % time spent in each bucket
    for i in range(len(wind_buckets)):
        wind_buckets[i] = wind_buckets[i] / total_points
    
    # Add temp band data to the dictionary                
    dict_result['wind_u5'] = wind_buckets[0]
    dict_result['wind_5_10'] = wind_buckets[1] 
    dict_result['wind_10_15'] = wind_buckets[2] 
    dict_result['wind_15_20'] = wind_buckets[3] 
    dict_result['wind_20_25'] = wind_buckets[4] 
    dict_result['wind_o25'] = wind_buckets[5] 
    
                   
    # Create a list to hold the temp band data
    temp_buckets = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]                 
                   
    # Add temp band data
    for i in range(len(np_temp)):
        bucket = which_bucket(np_temp[i], 'temp')
        temp_buckets[bucket] = temp_buckets[bucket] + 1
    
    # Find the total number of hours in the year
    total_points = sum(temp_buckets)
    
    # Check if any values did not fall into a bucket
    if total_points != 8760:
        print('ploc', ploc, ' total_temp_points: ', total_points)
    
    # Calculate the % time spent in each bucket
    for i in range(len(temp_buckets)):
        temp_buckets[i] = temp_buckets[i] / total_points
    
    # Add temp band data to the dictionary                
    dict_result['temp_u0'] = temp_buckets[0]
    dict_result['temp_0_10'] = temp_buckets[1] 
    dict_result['temp_10_20'] = temp_buckets[2] 
    dict_result['temp_20_30'] = temp_buckets[3] 
    dict_result['temp_30_40'] = temp_buckets[4] 
    dict_result['temp_o40'] = temp_buckets[5] 
    
    
    # Add wind power factor data to the dictionary
    wind_power_density = 0
    
    for i in range(len(np_temp)):
        wind_power_density = wind_power_density + WindPowerDensity(np_wind_speed[i], np_temp[i], np_press[i], np_dew_point[i])
    
    dict_result['wind_power_density'] = wind_power_density / len(np_temp)
    
    # Convert all the results to a pandas dataframe
    df = pd.DataFrame([dict_result])
    
    return df


In [10]:
dset_names = \
['air_temperature',
 'alpha',
 'aod',
 'asymmetry',
 'cld_opd_dcomp',
 'cld_reff_dcomp',
 'clearsky_dhi',
 'clearsky_dni',
 'clearsky_ghi',
 'cloud_press_acha',
 'cloud_type',
 'dew_point',
 'dhi',
 'dni',
 'ghi',
 'ozone',
 'relative_humidity',
 'solar_zenith_angle',
 'ssa',
 'surface_albedo',
 'surface_pressure',
 'total_precipitable_water',
 'wind_direction',
 'wind_speed']

def agg_on_idx(ploc_idx):
    # get mean of yearly aggregated fields
    dict_result = {}
    for field in dset_names:
        dset = f[field]
        dict_result['avg_' + field] = np.mean(dset[::1,ploc_idx] / dset.attrs['psm_scale_factor'])

    # get std dev of yearly aggregated fields
    for field in dset_names:
        dset = f[field]
        dict_result['std_' + field] = np.std(dset[::1,ploc_idx] / dset.attrs['psm_scale_factor'])

    df = pd.DataFrame([dict_result])
    return df

def get_variables_on_idx(ploc_idx):
    """
    Helper function to calculate wind and temperature bands and wind power density for a single plant.
    :ploc: tuple containing the latitude and longitude of the desired power plant
    :return: pandas dataframe containing wind and temperature bands and wind power density for a single plant    
    """
    
    # Create file objects for accessing needed datasets in the HDF5 file
    dset_coords = f['coordinates'][...]
    dset_temp = f['air_temperature']
    dset_press = f['surface_pressure']
    dset_dew_point = f['dew_point']
    dset_wind_speed = f['wind_speed']
    
    # Load the needed data into 1d numpy arrays
    np_temp = dset_temp[:, ploc_idx]
    np_press = dset_press[:, ploc_idx]
    np_dew_point = dset_dew_point[:, ploc_idx]
    np_wind_speed = dset_wind_speed[:, ploc_idx]
    
    # Scale each array
    np_temp = np_temp[:] / dset_temp.attrs['psm_scale_factor']
    np_press = np_press[:] / dset_press.attrs['psm_scale_factor']
    np_dew_point = np_dew_point[:] / dset_dew_point.attrs['psm_scale_factor']
    np_wind_speed = np_wind_speed[:] / dset_wind_speed.attrs['psm_scale_factor']
    
    # Create an empty dictionary to hold the results
    dict_result = {}
    
    # Add the actual latitude and longitude of the weather station
    dict_result['adj_latitude'] = dset_coords[ploc_idx][0]
    dict_result['adj_longitude'] = dset_coords[ploc_idx][1]
       
    # Create a list to hold the wind band data
    wind_buckets = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    
    # Add wind band data
    for i in range(len(np_wind_speed)):
        bucket = which_bucket(np_wind_speed[i], 'wind')
        wind_buckets[bucket] = wind_buckets[bucket] + 1
                   
    # Find the total number of hours in the year
    total_points = sum(wind_buckets)
    
    # Check if any values did not fall into a bucket
    if total_points != 8760:
        print('ploc', ploc, ' total_wind_points: ', total_points)
                   
    # Calculate the % time spent in each bucket
    for i in range(len(wind_buckets)):
        wind_buckets[i] = wind_buckets[i] / total_points
    
    # Add temp band data to the dictionary                
    dict_result['wind_u5'] = wind_buckets[0]
    dict_result['wind_5_10'] = wind_buckets[1] 
    dict_result['wind_10_15'] = wind_buckets[2] 
    dict_result['wind_15_20'] = wind_buckets[3] 
    dict_result['wind_20_25'] = wind_buckets[4] 
    dict_result['wind_o25'] = wind_buckets[5] 
    
    # Create a list to hold the temp band data
    temp_buckets = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]                 
                   
    # Add temp band data
    for i in range(len(np_temp)):
        bucket = which_bucket(np_temp[i], 'temp')
        temp_buckets[bucket] = temp_buckets[bucket] + 1
    
    # Find the total number of hours in the year
    total_points = sum(temp_buckets)
    
    # Check if any values did not fall into a bucket
    if total_points != 8760:
        print('ploc', ploc, ' total_temp_points: ', total_points)
    
    # Calculate the % time spent in each bucket
    for i in range(len(temp_buckets)):
        temp_buckets[i] = temp_buckets[i] / total_points
    
    # Add temp band data to the dictionary                
    dict_result['temp_u0'] = temp_buckets[0]
    dict_result['temp_0_10'] = temp_buckets[1] 
    dict_result['temp_10_20'] = temp_buckets[2] 
    dict_result['temp_20_30'] = temp_buckets[3] 
    dict_result['temp_30_40'] = temp_buckets[4] 
    dict_result['temp_o40'] = temp_buckets[5] 
    
    
    # Add wind power factor data to the dictionary
    wind_power_density = 0
    
    for i in range(len(np_temp)):
        wind_power_density = wind_power_density + WindPowerDensity(np_wind_speed[i], np_temp[i], np_press[i], np_dew_point[i])
    
    dict_result['wind_power_density'] = wind_power_density / len(np_temp)
    
    # Convert all the results to a pandas dataframe
    df = pd.DataFrame([dict_result])
    return df


In [ ]:
# Get coordinates in USA only (2018392 items -> 546219 items)
dset_names = \
['air_temperature',
 'alpha',
 'aod',
 'asymmetry',
 'cld_opd_dcomp',
 'cld_reff_dcomp',
 'clearsky_dhi',
 'clearsky_dni',
 'clearsky_ghi',
 'cloud_press_acha',
 'cloud_type',
 'dew_point',
 'dhi',
 'dni',
 'ghi',
 'ozone',
 'relative_humidity',
 'solar_zenith_angle',
 'ssa',
 'surface_albedo',
 'surface_pressure',
 'total_precipitable_water',
 'wind_direction',
 'wind_speed']

meta = pd.DataFrame(f['meta'][...])

USA = meta.loc[meta['country'] == b'United States'] # Note .h5 saves strings as bit-strings
USA.head()

# Keep 'elevation' and 'state' as well. They may be useful later.
df_coord_usa = USA[['latitude', 'longitude', 'elevation', 'state']].copy()
df_coord_usa.shape
df_coord_usa = df_coord_usa.reset_index()
df_coord_usa.head(50)

total_no_str = str(len(df_coord_usa.index))
print (total_no_str + ' coordinates in the US')

def valuation_formula(idx):
    print(str(idx) + ' out of ' + total_no_str)
    
    df_idx = pd.DataFrame(df_coord_usa.iloc[2]).transpose()
    df_idx = df_idx.reset_index().drop(['level_0'], axis=1)
    df_temp = agg_on_idx(df_coord_usa.index[2])
    df_temp2 = get_variables_on_idx(df_coord_usa.index[2])
    df_temp.head()

    df_all = pd.concat([df_idx, df_temp, df_temp2], axis=1)

    return df_all

df_coord_usa_new = df_coord_usa.apply(lambda row: valuation_formula(row.name), axis=1)

df_coord_usa_new.head()

df_coord_usa_new.to_csv('dataset/us_all_points.csv')

546219 coordinates in the US
0 out of 546219
1 out of 546219
2 out of 546219
3 out of 546219
4 out of 546219
5 out of 546219
6 out of 546219
7 out of 546219
8 out of 546219
9 out of 546219
10 out of 546219
11 out of 546219
12 out of 546219
13 out of 546219
14 out of 546219
15 out of 546219
16 out of 546219
17 out of 546219
18 out of 546219
19 out of 546219
20 out of 546219
21 out of 546219
22 out of 546219
23 out of 546219
24 out of 546219
25 out of 546219
26 out of 546219
27 out of 546219
28 out of 546219
29 out of 546219
30 out of 546219
31 out of 546219
32 out of 546219
33 out of 546219
34 out of 546219
35 out of 546219
36 out of 546219
37 out of 546219
38 out of 546219
39 out of 546219
40 out of 546219
41 out of 546219
42 out of 546219
43 out of 546219
44 out of 546219
45 out of 546219
46 out of 546219
47 out of 546219
48 out of 546219
49 out of 546219
50 out of 546219
51 out of 546219
52 out of 546219
53 out of 546219
54 out of 546219
55 out of 546219
56 out of 546219
57 out of 54